In [6]:
import pandas as pd
from mip import *

students_df = pd.read_csv('resource//students.csv')
cars_df = pd.read_csv('resource//cars.csv')

# 学生の乗車グループ分け問題（0-1整数計画問題）のインスタンス作成
m = Model()

# リスト
# 学生のリスト
S = students_df['student_id'].to_list()
# 車のリスト
C = cars_df['car_id'].to_list()
# 学年のリスト
G = [1, 2, 3, 4]
# 学生と車のペアのリスト
SC = [(s, c) for s in S for c in C]
# 免許を持っている学生のリスト
S_license = students_df[students_df['license'] == 1]['student_id']
# 学年が  g  の学生のリスト
S_g = {g: students_df[students_df['grade'] == g]['student_id'] for g in G}
# 男性と女性のリスト
S_male = students_df[students_df['gender'] == 0]['student_id']
S_female = students_df[students_df['gender'] == 1]['student_id']

# 定数
# 車の乗車定員
U = cars_df['capacity'].to_list()

# 変数
# 学生をどの車に割り当てるかを変数として定義
x = m.add_var_tensor((len(S),len(C)), name='x', var_type=BINARY)

# 制約
# (1) 各学生を１つの車に割り当てる
for s in S:
    m += xsum([x[s, c] for c in C]) == 1

# (2) 法規制に関する制約：各車には乗車定員より多く乗ることができない
for c in C:
    m += xsum([x[s, c] for s in S]) <= U[c]

# (3) 法規制に関する制約：各車にドライバーを1人以上割り当てる
for c in C:
    m += xsum([x[s, c] for s in S_license]) >= 1

# (4) 懇親を目的とした制約: 各車に各学年の学生を１人以上割り当てる
for c in C:
    for g in G:
        m += xsum([x[s, c] for s in S_g[g]]) >= 1

# (5) 各車に男性を1人以上割り当てる
for c in C:
    m += xsum([x[s, c] for s in S_male]) >= 1

# (6) 各車に女性を1人以上割り当てる
for c in C:
    m += xsum([x[s, c] for s in S_female]) >= 1

# 最適化後に利用するデータを返却
#return {'prob': m, 'variable': {'x': x}, 'list': {'S': S, 'C': C}}

# 最適化問題を解くメソッド
# 問題を解く
status = m.optimize()

# 最適化結果を格納

car2students = {c: [s for s in S if x[s, c].x == 1] for c in C}
student2car = {s: c for c, ss in car2students.items() for s in ss}
solution_df = pd.DataFrame(list(student2car.items()), columns=['student_id', 'car_id'])

In [7]:
status

<OptimizationStatus.OPTIMAL: 0>

In [8]:
solution_df

,student_id,car_id
0,5,0
1,6,0
2,11,0
3,12,0
4,0,1
5,7,1
6,10,1
7,17,1
8,8,2
9,9,2
